
### In this notebook, after loading the harmonized file (generated in 0.generate-feature-select-CRISPR-profile.ipynb) and '../01.create-annotations/output/gene_chromosome_map.tsv' , we append the Metadata_Chromosome and Metadata_Chromosome_arm columns to the data frame. 
### then, using the Recursion_U2OS_expression_data.csv provided through Recursion, we generate the correct arm profile. (https://www.biorxiv.org/content/10.1101/2023.04.15.537038v1 )

in Recursion_U2OS_expression_data.csv. they  uses -3 as a cutoff for calling a gene expressed and the value will be -10,000 if there were zero reads in the raw data. 

In [1]:
import pandas as pd
import numpy as np
import warnings
import utils
warnings.filterwarnings('ignore')

In [2]:
file_name="cc_notadjusted"
PCA = True

In [3]:
# Read Profiles 
df_crispr_raw = pd.read_parquet(f"../jump-CRISPR-data/mad_int_featselect_harmony.parquet")
# df_crispr_raw = pd.read_parquet(f"../jump-CRISPR-data/crispr_harmonized_no_sphering_profiles.parquet")
# df_crispr_raw = pd.read_parquet(f"../jump-CRISPR-data/{file_name}.parquet")
df_crispr_raw.shape

(51185, 872)

In [4]:
# Read metadata 
df_chrom = pd.read_csv('../01.create-annotations/output/gene_chromosome_map.tsv', sep='\t', dtype=str)
df_an = pd.read_csv('../jump-CRISPR-data/crispr.csv.gz')

In [5]:
# Preprocess dataframe
df_crispr = utils.remove_nan_features(df_crispr_raw)
df_crispr = utils.annotate_gene(df_crispr, df_an)
df_crispr = utils.annotate_chromosome(df_crispr, df_chrom)

df_no_arm = df_crispr[df_crispr['Metadata_Chromosome'].isna()].reset_index(drop=True)
df_crispr = utils.sort_on_chromosome(df_crispr)

Removed nan features: []


In [6]:
if PCA:
    df_crispr = pd.concat([df_no_arm, df_crispr], axis=0, ignore_index=True)
    df_crispr = utils.transform_data(df_crispr) # Perform PCA
    # df_crispr = utils.treatment_level(df_crispr, 'Metadata_JCP2022') # 
    df_crispr.columns = df_crispr.columns.astype(str)
    print(df_crispr.shape)
    df_crispr.to_parquet(f"../jump-CRISPR-data/{file_name}_PCA.parquet", index=False)

(51185, 414)


## Arm position correction from Recursion's U2OS expression data

In [7]:
df_exp = pd.read_csv("../jump-CRISPR-data/Recursion_U2OS_expression_data.csv")

In [8]:
# df_exp = df_exp[df_exp['zfpkm']>-1000].reset_index(drop=True)
df_exp = df_exp.assign(Metadata_arm=df_exp.gene.map(dict(zip(df_crispr.Metadata_Symbol, df_crispr.Metadata_arm))))
df_exp = df_exp.dropna(subset='Metadata_arm')

In [9]:
unexpressed_genes = df_exp[df_exp['zfpkm']<-3]['gene'].unique()
arm_to_include = df_exp[df_exp['zfpkm']<-3].groupby('Metadata_arm')['gene'].nunique() > 20
df_exp[df_exp['zfpkm']<-3].groupby('Metadata_arm')['gene'].nunique()

Metadata_arm
10p     10
10q     63
11p     44
11q     80
12p     27
12q     63
13q     42
14q     52
15q     49
16p     25
16q     34
17p     31
17q     66
18p      4
18q     20
19p     39
19q     53
1p     113
1q      81
20p     15
20q     24
21q     16
22q     29
2p      41
2q     113
3p      39
3q      63
4p      24
4q      80
5p      16
5q      62
6p      64
6q      54
7p      30
7q      58
8p      32
8q      38
9p      18
9q      50
Xp      27
Xq      55
Yp       3
Yq       5
Name: gene, dtype: int64

In [10]:
feature_cols = df_crispr.filter(regex="^(?!Metadata_)").columns
feature_cols_unexpressed = [feat+ '_unexpressed' for feat in feature_cols]

df_unexpressed_mean = df_crispr[
                            df_crispr['Metadata_Symbol'].isin(unexpressed_genes)
                        ].groupby('Metadata_arm')[feature_cols].mean()[
                            arm_to_include # filter for arms to include
                        ]

df_unexpressed_mean = df_crispr.merge(df_unexpressed_mean, 
                                        on='Metadata_arm', 
                                        how='left',
                                        suffixes=('', '_unexpressed'))[feature_cols_unexpressed]

In [11]:
df_unexpressed_mean = df_unexpressed_mean.fillna(0)
df_unexpressed_mean.columns = feature_cols

In [12]:
df_crispr[feature_cols] = df_crispr[feature_cols] - df_unexpressed_mean[feature_cols]

In [13]:
if not PCA:
    col = df_no_arm.columns
    df_crispr = pd.concat([df_no_arm, df_crispr[col]], axis=0, ignore_index=True)

In [14]:
print(df_crispr.shape)
if PCA:
    df_crispr.to_parquet(f"../jump-CRISPR-data/{file_name}_PCA_corrected.parquet", index=False)
df_crispr.to_parquet(f"../jump-CRISPR-data/{file_name}_corrected.parquet", index=False)

(51185, 414)
